In [19]:
from google.cloud import bigquery

client = bigquery.Client(project="prod-organize-arizon-4e1c0a83")

query = """
WITH rv AS (

SELECT DISTINCT

CASE
    WHEN p.partyaffiliation = 'DEM' THEN 'DEM'
    WHEN p.partyaffiliation = 'REP' THEN 'GOP'
    ELSE '3RD'
  END AS party
, COUNT(DISTINCT p.dwid) AS RV

FROM `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__person` AS p

INNER JOIN  `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS d
  ON p.dwid = d.dwid

WHERE d.state = 'AZ'
AND p.state = 'AZ'
AND p.voterstatus = 'active'
AND d.statehousedistrict = '19'

GROUP BY 1
ORDER BY 1
)

, lv AS(

SELECT DISTINCT

CASE
    WHEN p.partyaffiliation = 'DEM' THEN 'DEM'
    WHEN p.partyaffiliation = 'REP' THEN 'GOP'
    ELSE '3RD'
  END AS party
, COUNT(DISTINCT p.dwid) AS LV

FROM `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__person` AS p

INNER JOIN  `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS d
  ON p.dwid = d.dwid

LEFT JOIN `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__models` AS m
  ON d.dwid = m.dwid

WHERE d.state = 'AZ'
AND p.state = 'AZ'
AND p.voterstatus = 'active'
AND d.statehousedistrict = '19'
AND m.catalistmodel_voteprop2026 >= 70

GROUP BY 1
ORDER BY 1
)

SELECT
r.party
, r.rv - l.lv AS av
, l.lv
FROM rv AS r

LEFT JOIN lv AS l
  ON r.party = l.party
"""

df = client.query(query).to_dataframe()
df.head()

,party,av,lv
0,GOP,24264,41388
1,3RD,33486,18674
2,DEM,13084,22040


In [20]:

import altair as alt
import pandas as pd

party_colors = {
    "DEM": "#1f77b4",   # Dem blue
    "GOP": "#d62728",   # GOP red
    "3RD": "#7f7f7f"    # neutral gray
}

alt.renderers.enable("default")

RendererRegistry.enable('default')

In [23]:
av_lv_stack = (
    alt.Chart(df)
    .transform_filter("datum.av + datum.lv > 0")
    .transform_fold(["lv", "av"], as_=["segment", "count"])
    .transform_calculate(
      total_registered="datum.av + datum.lv"
    )
    .mark_bar()
    .encode(
        y=alt.Y("party:N", title="Party"),
        x=alt.X("count:Q", stack="zero", title="Voters"),
        color=alt.condition(
            "datum.segment === 'lv'",
            alt.Color(
                "party:N",
                title="Party (LV)",
                scale=alt.Scale(
                    domain=list(party_colors.keys()),
                    range=list(party_colors.values()),
                ),
            ),
            alt.value("#d0d0d0"),  # gray for AV remainder
        ),
        tooltip=[
            "party:N",
            alt.Tooltip("lv:Q", format=",", title="likely voters"),
            alt.Tooltip("av:Q", format=",", title="other registered voters"),
            alt.Tooltip("total_registered:Q", format=",", title="total registered voters"),
        ],
    )
)

av_lv_stack



alt.Chart(...)

In [24]:
av_lv_stack = av_lv_stack.interactive()
av_lv_stack.save("av_lv_stack.html")
